In [ ]:
import gc
import datetime
import logging

import numpy as np
import pandas as pd
import tensorflow as tf
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from ultralytics import YOLO

from utils.models import Detector, EmbeddingModel
from utils import common, tal, losses, logger, gradNorm
from utils.dataset import DecklistDataset


def make_adamw(model, lr=1e-4, momentum=0.9, decay=0.01):
    g = [], [], []  # optimizer parameter groups
    bn = tuple(v for k, v in nn.__dict__.items() if "Norm" in k)

    for param_name, param in model.named_parameters():
        if "bias" in param_name:  # bias (no decay)
            g[2].append(param)
        elif isinstance(param, bn):  # weight (no decay)
            g[1].append(param)
        else:  # weight (width decay)
            g[0].append(param)

    optimizer = torch.optim.AdamW(
        g[2],
        lr=lr,
        betas=(momentum, 0.999),
        weight_decay=0.0,
    )
    optimizer.add_param_group({"params": g[0], "weight_decay": decay})
    optimizer.add_param_group({"params": g[1], "weight_decay": 0.0})

    return optimizer


def make_dataset(df_path, deck_size):
    X_train = pd.read_csv(df_path)
    id_list = X_train["id"].tolist()
    prefix = "datasets/card_images_small/"
    id_list = list(map(lambda x: prefix + str(x) + ".jpg", id_list))
    card_type = list(
        map(lambda x: x.lower().startswith("pendulum"), X_train["type"].tolist())
    )
    return DecklistDataset(id_list, card_type, deck_size)

2025-04-03 07:54:15.232158: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-03 07:54:15.232228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-03 07:54:15.233308: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-03 07:54:15.240853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-03 07:54:16.171796: W tensorflow/compiler/tf2

In [9]:
# prepare student model
pretrained_model = YOLO("weights/yolov8n_detector.pt")
pre_model_dict = pretrained_model.model.model.state_dict()

student_model = Detector()
model_dict = student_model.state_dict()

# from checkpoint
# pre_model_dict = torch.load("last.pt")
# student_model.load_state_dict(pre_model_dict)

for name, param in student_model.named_parameters():
    if "dfl" in name:
        param.requires_grad_(False)

    if True:
        if "embedding_layers" in name:
            param.requires_grad_(True)
        else:
            param.requires_grad_(False)
    else:
        param.requires_grad_(False if "dfl" in name else True)

In [10]:
for key in pre_model_dict.keys():
    new_key = f"layer{key}"
    model_dict[new_key] = pre_model_dict[key].clone().detach()
student_model.load_state_dict(model_dict)
student_model = student_model.cuda()

In [11]:
for name, param in student_model.named_parameters():
    print(name, param.requires_grad, param.grad_fn)

layer0.conv.weight False None
layer0.bn.weight False None
layer0.bn.bias False None
layer1.conv.weight False None
layer1.bn.weight False None
layer1.bn.bias False None
layer2.cv1.conv.weight False None
layer2.cv1.bn.weight False None
layer2.cv1.bn.bias False None
layer2.cv2.conv.weight False None
layer2.cv2.bn.weight False None
layer2.cv2.bn.bias False None
layer2.m.0.cv1.conv.weight False None
layer2.m.0.cv1.bn.weight False None
layer2.m.0.cv1.bn.bias False None
layer2.m.0.cv2.conv.weight False None
layer2.m.0.cv2.bn.weight False None
layer2.m.0.cv2.bn.bias False None
layer3.conv.weight False None
layer3.bn.weight False None
layer3.bn.bias False None
layer4.cv1.conv.weight False None
layer4.cv1.bn.weight False None
layer4.cv1.bn.bias False None
layer4.cv2.conv.weight False None
layer4.cv2.bn.weight False None
layer4.cv2.bn.bias False None
layer4.m.0.cv1.conv.weight False None
layer4.m.0.cv1.bn.weight False None
layer4.m.0.cv1.bn.bias False None
layer4.m.0.cv2.conv.weight False None
la